In [38]:
import os, sys
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity
import importlib
importlib.reload(spot)
importlib.reload(roi_prop)
importlib.reload(intensity)

<module 'easi_fish.intensity' from 'D:\\SWAP\\code\\easifish-proc\\data_processing\\..\\easi_fish\\intensity.py'>

In [39]:
### Specify directories to data files
input_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs"
output_dir = os.path.join(input_dir, "testfull-aug5")

# fixed image (directory - n5 format)
img_dir = os.path.join(input_dir, "r2\\export_sigma3.n5")
# segmentation mask (tif format accepted here)
lb_dir  = os.path.join(input_dir, "r2\\segmentation\\lt171_gene_4tile_r2-c3.tif" ) 
# spot dir 
spot_dir = os.path.join(input_dir, "spots_pooled") # pool spots together; warpped and fixed
# bad roi
bad_roi = os.path.join(output_dir, "bad_roi_list.npy")
### end input

# output
out_roi = os.path.join(output_dir, "roi_all.csv") 
out_goodroi = os.path.join(output_dir, "roi.csv") 
out_spots = os.path.join(output_dir, "spots.csv")
out_intensity = os.path.join(output_dir, "intensity_c0_r2.csv")

In [40]:
### Load segmentation mask
### Get list of ROIs that are fully or partially outside the mask 
### Make sure to only include ROIs that are intact and in the overlapping regions across all rounds of FISH
lb=imread(lb_dir)
roi = np.max(lb)
print("number of ROIs in total:",roi)

bad_roi_list=np.load(bad_roi)
print('number of bad ROIs:', len(bad_roi_list))

number of ROIs in total: 13758
number of bad ROIs: 892


In [41]:
### Extract ROI properties
print(lb.shape)
lb = lb[500:600,500:600,500:600]
# lb.shape # z, x,y 

(1019, 915, 916)


In [42]:
%%time
### Extract ROI properties
df = roi_prop.roi_prop(lb)

C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\skimage\measure\_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 2022777011  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 10  Error-roundoff 1e-14  _one-merge 7.2e-14  _near-inside 3.6e-13
  Visible-distance 2.1e-14  U-max-coplanar 2.1e-14  Width-outside 4.1e-14
  _wide-facet 1.2e-13  _maxoutside 8.2e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p1(v4):     0     1     0
- p8(v3):     3     2     0
- p25(v2):    10     0     0
- p0(v1):     0     0     0

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.

CPU times: total: 47.1 s
Wall time: 2.61 s


C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\skimage\measure\_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 2022777011  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  4  Error-roundoff 4.3e-15  _one-merge 3e-14  _near-inside 1.5e-13
  Visible-distance 8.5e-15  U-max-coplanar 8.5e-15  Width-outside 1.7e-14
  _wide-facet 5.1e-14  _maxoutside 3.4e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p1(v4):     0     0     2
- p7(v3):     1     0     4
- p3(v2):     1     0     0
- p0(v1):     0     0     1

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet. 

,roi,z,y,x,area,minor_axis_length,major_axis_length,eccentricity,solidity
0,1.0,41.229921,31.799505,19.451946,79.007208,3.882587,26.592798,0.146001,0.787422
1,2.0,3.875844,23.812108,1.555209,265.993896,10.753584,24.829100,0.433104,0.896108
2,3.0,38.416707,41.567800,41.977212,443.649024,19.518692,24.459833,0.797990,0.937113
3,4.0,16.930890,36.248539,16.363112,1349.432448,26.958377,35.246973,0.764842,0.897983
4,5.0,18.313163,31.440374,43.722799,365.797152,12.974855,30.638046,0.423488,0.902236
...,...,...,...,...,...,...,...,...,...
57,58.0,40.003918,30.265155,0.308247,17.241168,3.235265,15.667296,0.206498,0.729323
58,59.0,7.900000,1.115774,1.490208,59.721984,8.114660,17.207684,0.471572,0.910569
59,60.0,1.291777,2.181560,39.471777,151.526760,9.737822,26.217210,0.371429,0.930169
60,61.0,41.422500,45.540000,10.120000,0.710976,0.000000,5.477226,0.000000,inf


In [43]:
df

,roi,z,y,x,area,minor_axis_length,major_axis_length,eccentricity,solidity
0,1.0,41.229921,31.799505,19.451946,79.007208,3.882587,26.592798,0.146001,0.787422
1,2.0,3.875844,23.812108,1.555209,265.993896,10.753584,24.829100,0.433104,0.896108
2,3.0,38.416707,41.567800,41.977212,443.649024,19.518692,24.459833,0.797990,0.937113
3,4.0,16.930890,36.248539,16.363112,1349.432448,26.958377,35.246973,0.764842,0.897983
4,5.0,18.313163,31.440374,43.722799,365.797152,12.974855,30.638046,0.423488,0.902236
...,...,...,...,...,...,...,...,...,...
57,58.0,40.003918,30.265155,0.308247,17.241168,3.235265,15.667296,0.206498,0.729323
58,59.0,7.900000,1.115774,1.490208,59.721984,8.114660,17.207684,0.471572,0.910569
59,60.0,1.291777,2.181560,39.471777,151.526760,9.737822,26.217210,0.371429,0.930169
60,61.0,41.422500,45.540000,10.120000,0.710976,0.000000,5.477226,0.000000,inf


In [30]:
%%time
lb_stat_v2 = roi_prop.roi_prop_v2(lb)

0.01499485969543457
0.04398989677429199
CPU times: total: 46.9 ms
Wall time: 44 ms


In [31]:
lb_stat_v2

,centroid-0,centroid-1,centroid-2,area,roi,z,y,x
0,98.166479,69.129359,42.286839,79.007208,362,41.229921,31.799505,19.451946
1,9.228199,51.765453,3.380889,265.993896,1314,3.875844,23.812108,1.555209
2,91.468349,90.364784,91.254808,443.649024,2167,38.416707,41.567800,41.977212
3,40.311644,78.801172,35.571984,1349.432448,2944,16.930890,36.248539,16.363112
4,43.602770,68.348639,95.049563,365.797152,3254,18.313163,31.440374,43.722799
...,...,...,...,...,...,...,...,...
57,95.247423,65.793814,0.670103,17.241168,11418,40.003918,30.265155,0.308247
58,18.809524,2.425595,3.239583,59.721984,12209,7.900000,1.115774,1.490208
59,3.075660,4.742522,85.808211,151.526760,12387,1.291777,2.181560,39.471777
60,98.625000,99.000000,22.000000,0.710976,12406,41.422500,45.540000,10.120000


In [20]:
%%time
xyz = []
for item in lb_stat:
    xyz.append(item.centroid)
xyz = np.asarray(xyz)
# xyz

CPU times: total: 15.6 ms
Wall time: 14 ms


In [21]:
lb_stat

In [8]:
lb_id = np.unique(lb[lb != 0])

In [9]:
lb_id

array([  362,  1314,  2167,  2944,  3254,  3346,  4186,  4906,  4920,
        5130,  5360,  5364,  5670,  5673,  6142,  6270,  6437,  6529,
        6597,  6711,  6727,  6743,  6822,  7124,  7239,  7368,  7421,
        7626,  7792,  7925,  7951,  7955,  8066,  8202,  8289,  8386,
        8390,  8497,  8829,  8832,  8869,  8966,  9142,  9306,  9515,
        9778,  9802,  9811,  9942, 10114, 10287, 10409, 10427, 10464,
       10720, 10870, 11328, 11418, 12209, 12387, 12406, 13046],
      dtype=int32)

In [19]:
df

,roi,z,y,x,area,minor_axis_length,major_axis_length,eccentricity,solidity
0,1.0,363.489055,17.368291,73.106487,102.469416,8.682837,17.057071,0.509046,0.795172
1,2.0,125.553784,51.560521,396.296073,1692.922728,30.310127,42.301903,0.716519,0.836767
2,3.0,111.095382,104.351716,254.707325,1133.917848,25.492777,34.220391,0.744959,0.892238
3,4.0,55.515114,52.433029,164.453027,1471.898064,27.790082,35.438489,0.784178,0.899034
4,5.0,324.728038,91.253685,221.069806,1436.260392,28.336591,35.553770,0.797007,0.887821
...,...,...,...,...,...,...,...,...,...
13753,13754.0,206.419432,212.102989,10.116867,234.888696,13.263912,21.561534,0.615166,0.890799
13754,13755.0,252.961215,197.694189,313.463300,412.632696,15.868981,32.699733,0.485294,0.846490
13755,13756.0,265.635180,65.784569,393.811699,170.012136,12.697420,20.252999,0.626940,0.702276
13756,13757.0,193.609296,72.832144,206.992864,160.413960,11.973614,20.823803,0.574997,0.831414


In [6]:
### Remove bad ROIs and save the file
df_filtered = df.loc[~df['roi'].isin(bad_roi_list)]
df_filtered.roi = df_filtered.roi.astype(int)
df_filtered.to_csv(out_goodroi)

C:\Users\zeiss\AppData\Local\Temp\ipykernel_22484\4290796699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.roi = df_filtered.roi.astype(int)


In [22]:
df.to_csv(out_roi)

In [8]:
# ### Lipofuscin subtraction (optional)
# # Load spot files for lipofuscin subtraction
# channel_1=np.loadtxt(os.path.join(spot_dir, 'spots_c0.txt'), delimiter=',')
# channel_2=np.loadtxt(os.path.join(spot_dir, 'spots_c1.txt'), delimiter=',')
# # Identify lipofuscin spots
# c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# # Remove lipofuscin spots
# true_pos_c0 = np.delete(channel_1, pAind, axis=0)
# true_pos_c1 = np.delete(channel_2, pBind, axis=0)

### spot extraction and save file
spotcount = spot.spot_counts(lb, spot_dir, verbose=False)
spotcount.to_csv(out_spots)

Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c0.txt
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c0.txt line# 79374
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 59548
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 293582
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 313383
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 320791
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 321804
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 323138
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line#

In [9]:
### Intensity measurement and save file
df_intensity=intensity.measure_intensity(lb, img_dir, 'c0')
df_intensity.to_csv(out_intensity)

In [15]:
test = pd.read_csv(out_spots, index_col=0)
for col in test:
    print(np.max(test[col]))
test

930.0
1615.0
1115.0
66.0
115.0
278.0
106.0
32.0


,spots_r1_c0,spots_r1_c1,spots_r1_c2,spots_r1_c4,spots_r2_c0,spots_r2_c1,spots_r2_c2,spots_r2_c4
1,34.0,46.0,37.0,25.0,0.0,1.0,0.0,0.0
2,4.0,13.0,0.0,0.0,0.0,2.0,2.0,1.0
3,5.0,20.0,0.0,1.0,2.0,5.0,0.0,0.0
4,3.0,14.0,0.0,2.0,0.0,4.0,0.0,1.0
5,5.0,23.0,0.0,4.0,14.0,12.0,0.0,2.0
...,...,...,...,...,...,...,...,...
13754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13755,0.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0
13756,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0
13757,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
